# Import Modules

In [1]:
%load_ext autoreload
%autoreload
%pylab inline
import os
import glob
import json
import pandas as pd
import pickle

#from IPython import embed
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 100)

Populating the interactive namespace from numpy and matplotlib


In [2]:
from helper_functions import *

# Read and Process Excel Sheets (Organized by Fuel Type and Labeled as Burnable or Unburnable)

In [3]:
data_base_path = 'raw_data'

### Read the processed excel sheet with alerts and confusion matrix

In [4]:
excel_processed = pd.ExcelFile(os.path.join(data_base_path, 'GroundTruth_FuelTypes13_Labeled.xlsx'))

In [5]:
sheet_to_df_map_read = {}

In [6]:
features_to_use = ['feature01', 'feature02', 'feature03', 'feature04','feature05', 'feature06', 'feature07']

In [7]:
for sheet in excel_processed.sheet_names:
    df_read = pd.DataFrame(pd.read_excel(excel_processed, sheet))
    if sheet == 'Summary':
        sheet_to_df_map_read[sheet] = df_read
    else:            
        sheet_to_df_map_read[sheet] = df_read[['SheetName'] + features_to_use + ['Label']]
del sheet_to_df_map_read['Summary']

In [8]:
sheet_to_df_map_read.keys()
#sheet_to_df_map_read['Summary']

dict_keys(['Fuel01', 'Fuel02', 'Fuel03', 'Fuel04', 'Fuel05', 'Fuel06', 'Fuel07', 'Fuel08', 'Fuel09', 'Fuel10', 'Fuel11', 'Fuel12', 'Fuel13'])

In [9]:
sheet_to_df_map_read['Fuel01'].head(5)

,SheetName,feature01,feature02,feature03,feature04,feature05,feature06,feature07,Label
0,Fuel01,7.666242,7.72519,5.78626,0.988016,7.307230,2.1272,7.265493,Fuel01_Burnable
1,Fuel01,7.302745,4.90000,3.48000,-4.522060,7.828646,2.5600,3.512169,Fuel01_Burnable
2,Fuel01,8.382824,3.19000,6.60000,18.669410,8.753171,1.9800,6.596178,Fuel01_Burnable
3,Fuel01,7.594138,1.28000,7.10000,-3.049260,9.798903,0.5200,3.619166,Fuel01_Burnable
4,Fuel01,4.672622,2.75000,2.24000,5.593350,4.177140,0.6900,1.405831,Fuel01_Burnable


In [10]:
sheet_to_df_map_read['Fuel06'].head(5)

,SheetName,feature01,feature02,feature03,feature04,feature05,feature06,feature07,Label
0,Fuel06,7.792590,8.74809,18.67180,-16.579847,8.174539,3.52786,5.558446,Fuel06_Unburnable
1,Fuel06,7.549181,6.38168,4.47328,26.241729,5.226250,3.26453,1.367019,Fuel06_Unburnable
2,Fuel06,8.333343,4.97710,13.78630,12.614086,4.131958,1.14677,7.140907,Fuel06_Unburnable
3,Fuel06,8.629077,8.74809,5.45038,35.645320,5.304478,3.93004,4.344894,Fuel06_Unburnable
4,Fuel06,8.345312,5.98473,17.20610,4.930259,6.004266,3.76966,4.640471,Fuel06_Unburnable


### Define features and labels

In [11]:
features = ['feature01', 'feature02', 'feature03', 'feature04','feature05', 'feature06', 'feature07']
labels = ['Label', 'NewLabel', 'SheetName']
cols = labels + features

### Re-Label sheets and combine and prune them

In [12]:
sheet_to_df_map = re_label_data(sheet_to_df_map_read)
df_combined = combine_df_sheets(sheet_to_df_map, sorted(sheet_to_df_map.keys()), cols)
df_combined_orig = df_combined
len(df_combined)

1210

In [13]:
len(df_combined)
#df_combined.keys()

1210

# SVM Stuff

### Scikit-Learn Imports

In [14]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
#from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, average_precision_score
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_recall_curve, classification_report
from time import time

### Work with Multiple Features of Interest

In [15]:
features = ['feature01', 'feature02', 'feature03', 'feature04','feature05', 'feature06', 'feature07']

#### Get Features and Labels for Training and Testing

In [16]:
X, y = get_features_labels(df_combined, features_to_use)
features_train, features_test, labels_train, labels_test = train_test_split(X, y, test_size=0.2, random_state=42)
sheets_train, sheets_test, labels_train, labels_test = split_labels_sheets(labels_train, labels_test)
len(features_train), len(features_test), len(labels_train), len(labels_test)

(964, 241, 964, 241)

#### Create and Train the Model

In [17]:
########################## SVM #################################
# Training Time is significantly more with large values of C
clf = SVC(kernel="linear", class_weight = "balanced") #class_weight

### fit the classifier on the training features and labels
t0 = time()
clf.fit(features_train, labels_train)
print ("Training Time:", round(time()-t0, 3), "s")

Training Time: 0.508 s


#### Save the Model to a File

In [18]:
# save the model to disk
'svm_N1205_T080_C1 --> svm_N(number of total data)_T(percentage for training)_C(model parameter in model constructor, default 1)'
svm_model_file = 'models_trained/svm_N1205_T080_C1'
pickle.dump(clf, open(svm_model_file, 'wb'))

#### Load the Model from a File (A template...use specific model for specific pupose)

In [19]:
# load the model from disk

## Predict on the Train Data itself using SVM

In [20]:
clf = pickle.load(open('models_trained/svm_N1205_T080_C1', 'rb'))
t1 = time()
labels_svm_pred = clf.predict(features_train)
print ("Prediction Time:", round(time()-t1, 3), "s")
#print ('labels_svm_pred: {}'.format(labels_svm_pred))

accuracy = accuracy_score(labels_svm_pred, labels_train)
print('Accuracy Score: {}'.format(accuracy))

conf_mat = confusion_matrix(labels_train, labels_svm_pred, labels = [1, 0])
print('Confusion Matrix: \n{}'.format(conf_mat))

print('Classification Report: \n')
print(classification_report(labels_train, labels_svm_pred, labels=[1, 0]))

average_precision = average_precision_score(labels_train, labels_svm_pred)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Prediction Time: 0.018 s
Accuracy Score: 0.8786307053941909
Confusion Matrix: 
[[183  11]
 [106 664]]
Classification Report: 

              precision    recall  f1-score   support

           1       0.63      0.94      0.76       194
           0       0.98      0.86      0.92       770

    accuracy                           0.88       964
   macro avg       0.81      0.90      0.84       964
weighted avg       0.91      0.88      0.89       964

Average precision-recall score: 0.61


## Predict on the Test Data using SVM

In [ ]:
### use the trained classifier to predict labels for the test features using SVM predict
clf = pickle.load(open('models_trained/svm_N1205_T080_C1', 'rb'))
t1 = time()
labels_svm_pred = clf.predict(features_test)
print ("Prediction Time:", round(time()-t1, 3), "s")
#print ('labels_svm_pred: {}'.format(labels_svm_pred))

accuracy = accuracy_score(labels_svm_pred, labels_test)
print('Accuracy Score: {}'.format(accuracy))

conf_mat = confusion_matrix(labels_test, labels_svm_pred, labels = [1, 0])
print('Confusion Matrix: \n{}'.format(conf_mat))

print('Classification Report: \n')
print(classification_report(labels_test, labels_svm_pred, labels=[1, 0]))

average_precision = average_precision_score(labels_test, labels_svm_pred)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

"""
precision, recall, thresholds = precision_recall_curve(labels_test, pred)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
"""